In [ ]:
import pandas as pd #to read the csv file
import numpy as np #for numerical computations
import seaborn as sns #graphs purpose
import matplotlib.pyplot as plt #plot purpose
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import os
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score as acs
from sklearn.metrics import precision_recall_fscore_support as score
import seaborn as sns
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

In [ ]:
df=pd.read_csv('/kaggle/input/breast-cancer-wisconsin-data/data.csv')

In [ ]:
print(df.shape)

In [ ]:
sns.countplot(df['diagnosis'], palette='husl')

In [ ]:
df.info

In [ ]:
df = df.drop(['Unnamed: 32', 'id'], axis=1)

In [ ]:
print(df.shape)

In [ ]:
df.diagnosis.unique()

In [ ]:
df['diagnosis'].value_counts()

In [ ]:
df['diagnosis']=df['diagnosis'].map({'M':1,'B':0})

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(),annot=True)

In [ ]:
# Generate and visualize the correlation matrix
corr = df.corr().round(2)

# Mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set figure size
f, ax = plt.subplots(figsize=(20, 20))

# Define custom colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap
sns.heatmap(corr, mask=mask, cmap=cmap, vmin=-1, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

plt.tight_layout()

In [ ]:
corr = df.corr()

In [ ]:
#drop mean columns
cols = ['radius_mean', 
        'texture_mean', 
        'perimeter_mean', 
        'area_mean', 
        'smoothness_mean', 
        'compactness_mean', 
        'concavity_mean',
        'concave points_mean', 
        'symmetry_mean', 
        'fractal_dimension_mean']
df = df.drop(cols, axis=1)

In [ ]:
#drop perimeter and area columns
cols = ['perimeter_se',
        'perimeter_worst', 
        'area_se', 
        'area_worst']
df = df.drop(cols, axis=1)

In [ ]:
#drop the columns with lower correlation
cols = ['texture_se',
       'smoothness_se',
       'symmetry_se',
       'fractal_dimension_se']
df = df.drop(cols,axis=1)

In [ ]:
#drop the concavity and concave points 
cols = ['concavity_worst',
        'concavity_se', 
        'compactness_worst', 
        'compactness_se']
df = df.drop(cols, axis=1)

In [ ]:
#print remaining columns
df.columns

In [ ]:
#correlation matrix
corr = df.corr().round(2)
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(corr, mask=mask, cmap=cmap, vmin=-1, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)
plt.tight_layout()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics

In [ ]:
x = df.drop(['diagnosis'], axis=1)
y = df['diagnosis']

In [ ]:
x.shape,y.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test,y_train, y_test=train_test_split(x,y,test_size=0.3,random_state=40)

# > **Outliers**

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
lof= LocalOutlierFactor()
y_pred=lof.fit_predict(x)
y_pred[0:30]
#  1 = inlier
# -1 = outlier

In [ ]:
x_score= lof.negative_outlier_factor_
outlier_score= pd.DataFrame()
outlier_score["score"]=x_score

lofthreshold= -2.5
loffilter= outlier_score["score"]< lofthreshold
outlier_index= outlier_score[loffilter].index.tolist()

In [ ]:
plt.figure(figsize=(12,8.))
plt.scatter(x.iloc[outlier_index,0],x.iloc[outlier_index,4],color="blue",s=50,label="outliers")
plt.scatter(x.iloc[:,0],x.iloc[:,4],color="k",s=3,label="Data Points")

radius=(x_score.max()- x_score)/(x_score.max()-x_score.min())
outlier_score["radius"]=radius
plt.scatter(x.iloc[:,0],x.iloc[:,4],s=1000*radius,edgecolors="r",facecolors="none",label="outlier scores")
plt.legend();

In [ ]:
x = x.drop(outlier_index)
y = y.drop(outlier_index).values

In [ ]:
x.shape

# > **Building Model**

In [ ]:
#Take the response as Y as Predictors as X
x=df.drop(['diagnosis'],axis=1)
y=df['diagnosis']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=40)

# >Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()

x_train=ss.fit_transform(x_train)
x_test=ss.fit_transform(x_test)

# > **Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

model1=lr.fit(x_train,y_train)
prediction1=model1.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test,prediction1)
cm

In [ ]:
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
TP=cm[0][0]
TN=cm[1][1]
FN=cm[1][0]
FP=cm[0][1]
print('Testing Accuracy:',(TP+TN)/(TP+TN+FN+FP))

In [ ]:
metrics.plot_roc_curve(lr, x_test, y_test)

# > **Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc=DecisionTreeClassifier()
model2=dtc.fit(x_train,y_train)
prediction2=model2.predict(x_test)
cm2= confusion_matrix(y_test,prediction2)

cm2

In [ ]:
accuracy_score(y_test,prediction2)

In [ ]:
metrics.plot_roc_curve(dtc, x_test, y_test)

# > **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier()
model3 = rfc.fit(x_train, y_train)
prediction3 = model3.predict(x_test)
confusion_matrix(y_test, prediction3)

In [ ]:
accuracy_score(y_test, prediction3)

In [ ]:
metrics.plot_roc_curve(rfc, x_test, y_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, prediction1))

print(classification_report(y_test, prediction2))

print(classification_report(y_test, prediction3))


In [ ]:
key = ['LogisticRegression','DecisionTreeClassifier','RandomForestClassifier']
value = [LogisticRegression(), DecisionTreeClassifier(random_state=10), RandomForestClassifier(n_estimators=60, random_state=0)]
models = dict(zip(key,value))
models

In [ ]:
predicted =[]
for name,algo in models.items():
    model=algo
    model.fit(x_train,y_train)
    predict = model.predict(x_test)
    acc = accuracy_score(y_test, predict)
    predicted.append(acc)
    print(name,acc)

In [ ]:
plt.figure(figsize = (10,5))
sns.barplot(x = predicted, y = key, palette='pastel')

In [ ]:
lr = LogisticRegression(solver='lbfgs', max_iter=10000)
rs = []
acc = []
for i in range(1,25,1):
    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.2, random_state = i)    
    model_lr_rs = lr.fit(x_train, y_train)
    predict_values_lr_rs = model_lr_rs.predict(x_test)
    acc.append(accuracy_score(y_test, predict_values_lr_rs))
    rs.append(i)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(rs, acc, color ='blue')

for i in range(len(rs)):
    print(rs[i],acc[i])

In [ ]:
for i in range(0,24):
    if acc[i] > 0.98:
        print(acc[i])